<a href="https://colab.research.google.com/github/baleineindigo/Statistic/blob/main/%ED%99%95%EB%A5%A0%EB%A1%A0%EC%A0%81_%EC%84%A0%ED%98%95_%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
from sklearn.datasets import load_boston
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler


# GPU 설정 : 런타임 > 런타임 유형 변경 > GPU
# GPU 사용 가능한지 확인
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

## 1. 부트스트래핑

비모수적 추론. 특정 추정량의 확률적 성질을 수학적으로는 파악하기 힘든 상황에서 특히 유용. 주어진 데이터들을 원래의 모집단을 대표하는 독립 표본으로 가정하여 수집된 데이터들로부터 중복을 허용한 무작위 반복 추출을 하고 여기에서 얻은 통계량의 오차분산들을 활용하여 표본확률분포를 구한다.

회귀분석에 사용한 표본 데이터가 달라질 때 회귀분석의 결과는 어느 정도 영향을 받는지를 알기 위한 방법이다.

그러나 현실적으로는 데이터를 추가적으로 얻기가 힘들기 때문에 부트스트래핑 방법에서는 기존의 데이터를 재표본화(re-sampling)하는 방법을 선택한다. 재표본화는 기존의 $D$개의 데이터에서 다시 $D$개의 데이터를 선택하되 중복 선택도 가능하게 한다. (resampling with replacement) 이 경우 이론적으로는 $2^D$개의 새로운 표본 집단을 얻을 수 있다


In [ ]:
# 임의의 회귀식 생성
from sklearn.datasets import make_regression

X0, y, coef = make_regression(n_samples=100, n_features=1, noise=20,
                              coef=True, random_state=0)

# 표본 데이터로 회귀분석
dfX0 = pd.DataFrame(X0, columns=["X1"])
dfX = sm.add_constant(dfX0) # 상수항 추가
dfy = pd.DataFrame(y, columns=["y"])

model = sm.OLS(dfy, dfX)
result = model.fit()

In [ ]:
'''
다음으로 이 데이터에서 중복을 허락하여 N개의 데이터를 선택한 후 다시 회귀 분석을 한다.
여기에서는 이러한 회귀분석을 1,000번 반복한다. 
그 결과로써 1,000개의 다른 가중치 추정값이 나오게 된다.
'''
%%time
N = 1000
# w0,w1 1000개 무작위 추출
params_w0 = np.zeros(N)
params_w1 = np.zeros(N)
for i in range(N):
    idx = np.random.choice(len(dfy), len(dfy), replace=True) # X 무작위 추출
    #replace=True: 중복 허용
    dfX2 = dfX.iloc[idx, :]
    dfy2 = dfy.iloc[idx]
    r = sm.OLS(dfy2, dfX2).fit()
    params_w0[i] = r.params.const
    params_w1[i] = r.params.X1

In [ ]:
# 전체 가중치 추정값 집합을 히스토그램으로 나타내면 다음과 같다.
plt.figure(figsize=(8, 3))

ax1 = plt.subplot(121)
sns.distplot(params_w1, ax=ax1)
plt.axvline(params_w1.mean(), c='r')
plt.title("$\hat{w}_1$ 추정치 분포")

ax2 = plt.subplot(122)
sns.distplot(params_w0, ax=ax2)
plt.axvline(params_w0.mean(), c='r')
plt.title("$\hat{w}_0$ 추정치 분포")
plt.show()

In [ ]:
'''
추정치 분포의 평균과 분산은 다음과 같다. 
기울기의 추정치 분포는 평균이 42.92, 표준편차가 1.92이다. 
대부분의 데이터가 42.92±3.84 (x¯±2s) 사이에 있다. 
따라서 오차가 큰 경우에도 0 혹은 음수가 될 가능성은 적다.
'''
params_w1.mean(), params_w1.std()

In [ ]:
'''
그런데 상수항의 추정치 분포는 평균이 −1.67, 표준편차가 2.16이다. 
대부분의 데이터가 −1.67±4.32(−5.99∼2.65) 사이에 있기 때문에 0일 수도 있고 혹은 음수일 수도 있다.
'''
params_w0.mean(), params_w0.std()

In [ ]:
print(result.summary())
'''
이 결과를 statsmodels의 summary 메서드로 출력한 보고서와 비교해보면 
추정치의 표준편차와 비슷한 값이 std err이라는 이름으로 표시되어 있고 
x¯±2s로 추정한 구간과 비슷한 값이 [0.025      0.975] 열 아래에 표시되어 있다. 
예를 들어 상수항 추정치는 std err가 2.163 이고 [0.025      0.975]열 값이 −5.920∼2.663이다.


정규화된 모수 오차 w_hat/se를 검정 통계량으로 사용하면 
w_hat이 0 인지 아닌지에 대한 검정을 실시할 수 있다.

H0:wi=0(i=0,…,K−1)

만약 이 검정에 대한 유의 확률이 0에 가깝게 나온다면 위의 귀무가 설은 기각이므로 
wi 값이 0일 가능성은 적다.
하지만 유의 확률이 유의 수준을 넘는 큰 값이 나온다면 반대로 wi 값이 0일 가능성이 크다. 
즉, 해당 독립 변수는 종속 변수와 아무런 상관성이 없을 가능성이 있다는 뜻이다.

StatsModels summary 메서드가 출력하는 회귀분석 보고서에서 
std err로 표시된 열이 모형계수의 표준오차, 
t로 표시된 열이 단일 계수 t-검정의 검정 통계량, 그리고 P>|t|로 표시된 열이 유의확률을 뜻한다.

아래의 보고서에서는 첫번째 모형계수는 실제 값이 0일 가능성 
즉, 상수항을 가지지 않을 가능성이 높다고 할 수 있다.
'''


### 확률론적 선형 회귀모형¶
    확률론적 선형 회귀모형에서는 데이터가 확률 변수로부터 생성된 표본이라고 가정한다. 구체적인 가정은 다음과 같다.

    1) Linear and Normal Dstribution 선형 정규 분포 가정¶, $p(ϵ∣θ)=N(0,σ2)$

    선형 회귀분석의 기본 가정은 종속 변수 y가 독립 변수 x의 선형 조합으로 결정되는 기댓값과 고정된 분산 σ2을 가지는 가우시안 정규 분포라는 것이다.


    2) 외생성(Exogeneity) 가정¶, E[ϵ]=E[E[ϵ∣x]]=0

    오차 ϵ의 기댓값은 독립 변수 x의 크기에 상관없이 항상 0이라고 가정한다. 이를 외생성(Exogeneity) 가정이라고 한다. 
    외생성 가정으로부터와 오차 ϵ의 무조건부 기댓값이 0임을 증명할 수 있다.


    3) 조건부 독립 가정¶,Cov[ϵ]=E[ϵϵT]=diagonal matrix
    i번째 표본의 오차 ϵi와 j번째 표본의 오차 ϵj의 공분산 값이 x와 상관없이 항상 0이라고 가정한다.


    4) 등분산성 가정¶, Cov[ϵ]=E[ϵϵT]=σ2

    i번째 표본의 잡음 ϵi와 j번째 표본의 잡음 ϵj의 분산 값이 표본과 상관없이 항상 같다고 가정한다.
    잡음 벡터 ϵ의 공분산 행렬이 항등행렬 형태가 되어야 한다는 조건과 같다.


### MLE(MaximumLikleyhood Estimation)
모수적인 데이터 추정 방법으로서, 파라미터 세타로 구성된 확률밀도함수 P(x|세타)에서 관측된 표본 데이터 집합 x={x1,x2,x3...,xm}에서 세타를 추정하는 방법

즉, 모집단에서 추출된 표본을 가지고 분포를 추정하는 방법.

표본의 분포에 대하여 xi가 likelihood에 기여하는 정도.
liklihood Function ∏를 최대화하는 세타를 추정치 세타_hat으로 선정
P(x|세타)= ∏P(xk|세타)

일반적으로 MLE를 로그취해서 사용.


## 잔차의 분포
확률론적 선형 회귀모형에 따르면 회귀분석에서 생기는 잔차 e=y−w^Tx 도 정규 분포를 따른다. 다음과 같이 증명할 수 있다.

확률론적 선형 회귀모형의 잡음 ϵ와 잔차 e는 다음과 같은 관계를 가진다.

y_hat=Xw_hat=X*(X^T*X)^−1*X^T*y=H*y

이 행렬 H은 Hat 행렬 혹은 프로젝션(projection) 행렬 또는 영향도(influence) 행렬이라고 부르는 대칭 행렬이다.

e=y−y_hat=y−Hy=(I−H)y=My

이 행렬 M은 잔차(residual) 행렬이라고 부른다.

확률적 선형 회귀 모형의 가정을 적용하면,

e=My=M(Xw+ϵ)=MXw+Mϵ

그런데
MX=0 에서 e=Mϵ

즉, **잔차 e는 잡음 ϵ의 선형 변환(linear transform)**이다. 정규 분포의 선형 변환은 마찬가지로 정규 분포이므로 잔차도 정규 분포를 다른다.

In [ ]:
# 잔차의 정규성은 다음과 같이 정규성 검정을 통해 살펴볼 수도 있다.
test = sm.stats.omni_normtest(result.resid)
for xi in zip(['Chi^2', 'P-value'], test):
    print("%-12s: %6.3f" % xi)

for xi in zip(['Jarque-Bera', 'P-value', 'Skew', 'Kurtosis'], test):
    print("%-12s: %6.3f" % xi)

In [ ]:
'''
다음은 x값이 달라짐에 따라 잔차의 분포가 어떻게 바뀌는 지를 살펴보기 위한 것이다. 
x값이 달라져도 분포의 형태가 크게 바뀌지 않음을 알 수 있다.
'''
plt.plot(X0, result.resid, 'o',c="c")
plt.axhline(y=0, c='r')
plt.xlabel("X1")
plt.ylabel("Residual")
plt.show()

In [ ]:
## 단일 계수 t검정 
print(result.t_test("X1 = 40"))
'''
H0:w1=40
x1에 대한 계수를 40으로 한다고 해도 문제가 없음을 알 수 있다.

이 방법은 두 독립변수의 계수값을 비교할 때도 쓸 수 있다. 
범주형 독립변수의 범주값이 가지는 유의성을 판단하는데 유용하다.
'''

In [ ]:
# 특정파일을 가져오고 싶은 경우 다음과 같이 접근한다.
bst=load_boston()
# 데이터 프레임 형태로 가공
bst_data=pd.DataFrame(data=bst.data,columns=bst.feature_names)
bst_target=pd.DataFrame(data=bst.target,columns=["House_Price_Mediv"]) 
bst_all=pd.concat([bst_data,bst_target],axis=1)

In [ ]:
# CHAS 변수를 절편으로 하는 더미 회귀식 만들기
## 절편이 1인 model
feature_names = list(bst.feature_names)
feature_names.remove("CHAS")
feature_names=[name for name in feature_names]+["C(CHAS)"]
# print(feature_names)
dummy_model1=sm.OLS.from_formula("House_Price_Mediv ~ 0 +" + "+".join(feature_names),
                                 data=bst_all)
result=dummy_model1.fit()

In [ ]:
# Charles 강가에 위치했는지 유무에 따라 차이가 있는지
print(result.t_test("C(CHAS)[0.0] = C(CHAS)[1.0]"))
#  P>|t| 0.002 이므로 차이가 없다고 말할 수 없다. 즉, 차이가 있다.

### 회귀분석 F-검정
개별 개수가 아닌 전체 회귀 계수가 모두 의미가 있는지 확인하는 경우에는 다음과 같은 귀무 가설을 생각할 수 있다.

H0:w0=w1=⋯=wK−1=0   ==> 
이는 전체 독립 변수 중 의미를 가진 변수가 1개도 없다는 뜻이다. 

대부분의 경우, 이 귀무가설은 기각된다.(=의미있는 변수가 1개라도 존재함) 다만 유의 확률이 얼마나 작은가에 따라서 기각되는 정도가 달라진다. 유의 확률이 작으면 작을수록 더 강력하게 기각된 것이므로 더 의미가 있는 모형이라고 할 수 있다. 따라서 여러 모형의 유의 확률을 비교하여 어느 모형이 더 성능이 좋은가를 비교할 때 이 유의 확률을 사용한다. 이러한 검정을 Loss-of-Fit 검정 또는 **회귀분석 F-검정(regression F-test)**이라고 한다.



In [ ]:
## F-statistic = 400.3, P(F-statistic) = 2.21e-36 이므로 
print(result.summary())